# Processing all session data and predict subject emotion with LDA

## Set up

In [ ]:
import os
import glob
import time
import matplotlib.pyplot as plt
from matplotlib import rcParams

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from get_feature_response import *

# figure size in inches: 
# https://stackoverflow.com/questions/31594549/
# how-to-change-the-figure-size-of-a-seaborn-axes-or-figure-level-plot
rcParams['figure.figsize'] = 11.7,8.27

In [ ]:
CATEGORIES = {
    (0, 0) : "lvla",
    (0, 1) : "lvha",
    (1, 0) : "hvla",
    (1, 1) : "hvha"
}

min_record_len = 35

### Get preprocessed data

In [ ]:
%%capture 
start_time = time.time()

raw_path = "raw"
all_sessions = os.listdir(raw_path)

loaded_raws = []

for session in all_sessions:
    session_path = os.path.join(raw_path, session)
    raw = mne.io.Raw(session_path, preload=True)
    
    loaded_raws.append(raw)

In [ ]:
print("Took %ss to finish." % (time.time() - start_time))

### Get emotion response

In [ ]:
dataset_path = "/net2/expData/affective_eeg/mahnob_dataset/Sessions"
meta_data_path = "session.xml"
all_session_nums = os.listdir(dataset_path) # List of all session names

session_nums = [] # Sessions with bdf recordings

# Get current working directory to change back later
curr_dir = os.getcwd()

# From data manual, bdf file may not exist if "the trials is missing due to 
# technical difficulties" (pg 15).
# Skip all sessions with no bdf recordings
for session in all_session_nums:
    session_path = os.path.join(dataset_path, session)
    os.chdir(session_path)
    bdf_list = glob.glob("*.bdf")
    
    if (len(bdf_list) == 1):
        session_nums.append(session)
    
    elif (len(bdf_list) > 1):
        raise ValueError("Cannot handle multiple bdf files in one session.")

# Change back to notebook directory as a precaution
os.chdir(curr_dir)
print("Back to directory: ", os.getcwd())

response_list = []

for session in session_nums:
    xml_path = os.path.join(dataset_path, session, meta_data_path)
    resp = get_affect(xml_path)
    response_list.append(resp)

response_array = np.array(response_list)

In [ ]:
print("response_array shape:", np.shape(response_array))

## 1. Alpha+Beta band & 2.5 sec window

In [ ]:
FREQ_BANDS = ["alpha", "beta"]
CHANNELS = ["F3", "Fz", "F4", "FC1", "FC2"]
WINDOW = 2.5

### Feature extraction

__Warning: Long run time.__

In [ ]:
%%capture
start_time = time.time()

features_list = []
features_red_dim_list = []

for raw in loaded_raws:
    _features, _features_red_dim = bdf_to_features(raw=raw, 
                                                   duration=min_record_len,
                                                   freq_bands=FREQ_BANDS, 
                                                   chs=CHANNELS,
                                                   window=WINDOW)
    features_list.append(_features)
    features_red_dim_list.append(_features_red_dim)

features_array = np.array(features_list)
features_red_dim_array = np.array(features_red_dim_list)

In [ ]:
print("Took %ss to finish." % (time.time() - start_time))
print("features_list shape:", np.shape(features_list))
print("features_red_dim_list shape:", np.shape(features_red_dim_list))

### Prediction

#### LDA

In [ ]:
lda_v = LDA()
lda_v_1_scores = cross_val_score(lda_v, features_red_dim_list, response_array[:,0], cv=5)
print(lda_v_1_scores)

In [ ]:
lda_a = LDA()
lda_a_1_scores = cross_val_score(lda_a, features_red_dim_list, response_array[:,1], cv=5)
print(lda_a_1_scores)

#### SVC

In [ ]:
svc_v = SVC()
svc_v_1_scores = cross_val_score(svc_v, features_red_dim_list, response_array[:,0], cv=5)
print(svc_v_1_scores)

In [ ]:
svc_a = SVC()
svc_a_1_scores = cross_val_score(svc_a, features_red_dim_list, response_array[:,1], cv=5)
print(svc_a_1_scores)

## 2. Alpha+beta band & 0.5 sec window

In [ ]:
FREQ_BANDS = ["alpha", "beta"]
CHANNELS = ["F3", "Fz", "F4", "FC1", "FC2"]
WINDOW = 0.5

### Feature extraction

__Warning: Long run time.__

In [ ]:
%%capture
start_time = time.time()

features_list = []
features_red_dim_list = []

for raw in loaded_raws:
    _features, _features_red_dim = bdf_to_features(raw=raw, 
                                                   duration=min_record_len, 
                                                   freq_bands=FREQ_BANDS, 
                                                   chs=CHANNELS,
                                                   window=WINDOW)
    features_list.append(_features)
    features_red_dim_list.append(_features_red_dim)

features_array = np.array(features_list)
features_red_dim_array = np.array(features_red_dim_list)

In [ ]:
print("Took %ss to finish." % (time.time() - start_time))
print("features_list shape:", np.shape(features_list))
print("features_red_dim_list shape:", np.shape(features_red_dim_list))

### Prediction

#### LDA

In [ ]:
lda_v = LDA()
lda_v_2_scores = cross_val_score(lda_v, features_red_dim_list, response_array[:,0], cv=5)
print(lda_v_2_scores)

In [ ]:
lda_a = LDA()
lda_a_2_scores = cross_val_score(lda_a, features_red_dim_list, response_array[:,1], cv=5)
print(lda_a_2_scores)

#### SVC

In [ ]:
svc_v = SVC()
svc_v_2_scores = cross_val_score(svc_v, features_red_dim_list, response_array[:,0], cv=5)
print(svc_v_2_scores)

In [ ]:
svc_v = make_pipeline(StandardScaler(), SVC())
svc_a_2_scores = cross_val_score(svc_a, features_red_dim_list, response_array[:,1], cv=5)
print(svc_a_2_scores)

## 3. Alpha+beta+theta band & 2.5 sec window

In [ ]:
FREQ_BANDS = ["alpha", "beta", "theta"]
CHANNELS = ["F3", "Fz", "F4", "FC1", "FC2"]
WINDOW = 2.5

### Feature extraction

__Warning: Long run time.__

In [ ]:
%%capture
start_time = time.time()

features_list = []
features_red_dim_list = []

for raw in loaded_raws:
    _features, _features_red_dim = bdf_to_features(raw=raw, 
                                                   duration=min_record_len, 
                                                   freq_bands=FREQ_BANDS, 
                                                   chs=CHANNELS,
                                                   window=WINDOW)
    features_list.append(_features)
    features_red_dim_list.append(_features_red_dim)

features_array = np.array(features_list)
features_red_dim_array = np.array(features_red_dim_list)

In [ ]:
print("Took %ss to finish." % (time.time() - start_time))
print("features_list shape:", np.shape(features_list))
print("features_red_dim_list shape:", np.shape(features_red_dim_list))

### Prediction

#### LDA

In [ ]:
lda_v = LDA()
lda_v_3_scores = cross_val_score(lda_v, features_red_dim_list, response_array[:,0], cv=5)
print(lda_v_3_scores)

In [ ]:
lda_a = LDA()
lda_a_3_scores = cross_val_score(lda_a, features_red_dim_list, response_array[:,1], cv=5)
print(lda_a_3_scores)

#### SVC

In [ ]:
svc_v = make_pipeline(StandardScaler(), SVC())
svc_v_3_scores = cross_val_score(svc_v, features_red_dim_list, response_array[:,0], cv=5)
print(svc_v_3_scores)

In [ ]:
svc_v = make_pipeline(StandardScaler(), SVC())
svc_a_3_scores = cross_val_score(svc_a, features_red_dim_list, response_array[:,1], cv=5)
print(svc_a_3_scores)

## 4. Alpha+beta band & 2.5 sec window & more frontal channel

In [ ]:
FREQ_BANDS = ["alpha", "beta"]
CHANNELS = ["F3", "Fz", "F4", "AF3", "AF4"]
WINDOW = 2.5

### Feature extraction

__Warning: Long run time.__

In [ ]:
%%capture
start_time = time.time()

features_list = []
features_red_dim_list = []

for raw in loaded_raws:
    _features, _features_red_dim = bdf_to_features(raw=raw, 
                                                   duration=min_record_len, 
                                                   freq_bands=FREQ_BANDS, 
                                                   chs=CHANNELS,
                                                   window=WINDOW)
    features_list.append(_features)
    features_red_dim_list.append(_features_red_dim)

features_array = np.array(features_list)
features_red_dim_array = np.array(features_red_dim_list)

In [ ]:
print("Took %ss to finish." % (time.time() - start_time))
print("features_list shape:", np.shape(features_list))
print("features_red_dim_list shape:", np.shape(features_red_dim_list))

### Prediction

#### LDA

In [ ]:
lda_v = LDA()
lda_v_4_scores = cross_val_score(lda_v, features_red_dim_list, response_array[:,0], cv=5)
print(lda_v_4_scores)

In [ ]:
lda_a = LDA()
lda_a_4_scores = cross_val_score(lda_a, features_red_dim_list, response_array[:,1], cv=5)
print(lda_a_4_scores)

#### SVC

In [ ]:
svc_v = make_pipeline(StandardScaler(), SVC())
svc_v_4_scores = cross_val_score(svc_v, features_red_dim_list, response_array[:,0], cv=5)
print(svc_v_4_scores)

In [ ]:
svc_v = make_pipeline(StandardScaler(), SVC())
svc_a_4_scores = cross_val_score(svc_a, features_red_dim_list, response_array[:,1], cv=5)
print(svc_a_4_scores)

## Evaluation

The four models:
1. Alpha+Beta band & 2.5 sec window
2. Alpha+beta band & 0.5 sec window
3. Alpha+beta+theta band & 2.5 sec window
4. Alpha+beta band & 2.5 sec window & more frontal channel (replace FC1 and FC2 with AF3 and AF4)

### Valence

In [ ]:
# Adapted from https://matplotlib.org/stable/gallery/lines_bars_and_markers/barchart.html
labels = ['Model 1', 'Model 2', 'Model 3', 'Model 4']

lda_v_means = [np.mean(lda_v_1_scores), np.mean(lda_v_2_scores), 
               np.mean(lda_v_3_scores), np.mean(lda_v_4_scores)]
svc_v_means = [np.mean(svc_v_1_scores), np.mean(svc_v_2_scores), 
               np.mean(svc_v_3_scores), np.mean(svc_v_4_scores)]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, lda_v_means, width, label='LDA')
rects2 = ax.bar(x + width/2, svc_v_means, width, label='SVC')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('Average cross validation scores of LDA and SVC for valence prediction')
ax.set_xticks(x, labels)
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()

### Arousal

In [ ]:
# Adapted from https://matplotlib.org/stable/gallery/lines_bars_and_markers/barchart.html
labels = ['Model 1', 'Model 2', 'Model 3', 'Model 4']

lda_v_means = [np.mean(lda_a_1_scores), np.mean(lda_a_2_scores), 
               np.mean(lda_a_3_scores), np.mean(lda_a_4_scores)]
svc_v_means = [np.mean(svc_a_1_scores), np.mean(svc_a_2_scores), 
               np.mean(svc_a_3_scores), np.mean(svc_a_4_scores)]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, lda_v_means, width, label='LDA')
rects2 = ax.bar(x + width/2, svc_v_means, width, label='SVC')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Scores')
ax.set_title('Average cross validation scores of LDA and SVC for arousal prediction')
ax.set_xticks(x, labels)
ax.legend()

ax.bar_label(rects1, padding=3)
ax.bar_label(rects2, padding=3)

fig.tight_layout()

plt.show()